## **Introduction**  

**ETL(Extract, Transform, Load)** 작업은 **데이터 엔지니어** 역할에서 매우 중요한 과정입니다.  

- **Extract(추출)**: 여러 소스와 다양한 파일 형식에서 데이터를 가져오는 단계  
- **Transform(변환)**: 추출된 데이터를 **미리 정의된 설정**에 맞게 변환하는 단계  
- **Load(적재)**: 변환된 데이터를 **데이터베이스에 적재**하여 추가 처리에 사용되는 단계  

이 실습에서는 **ETL 작업을 직접 수행**하며 실습 경험을 쌓게 됩니다. 😊

## **Objectives (목표)**  

이 실습을 완료한 후, 다음을 수행할 수 있습니다:  

1️⃣ **CSV, JSON, XML 파일 읽기**  
2️⃣ 다양한 파일 형식에서 **필요한 데이터 추출**  
3️⃣ **데이터를 필요한 형식으로 변환**  
4️⃣ 변환된 데이터를 **RDBMS(Relational Database Management System)**에 **적재 가능한 형식으로 저장**  

## **Initial Steps (초기 단계)**  

ETL 프로세스의 첫 단계는 **기본 프로젝트 폴더(IDE의 default project folder)**에 **새 파일을 생성**하는 것입니다.

---

### 🔧 **파일 생성 방법:**

1️⃣ **메뉴 바**에서 **File 탭**을 클릭합니다.  
2️⃣ **New File**을 선택하여 **새 파일**을 만듭니다.  
3️⃣ **파일 이름**을 **`etl_code.py`**로 지정하고, 다음 경로에 **저장**합니다:  
   **`\home\project\etl_code.py`**

---

이제 Python 파일이 준비되었으니 ETL 코드를 작성할 수 있습니다! 😊

Project Source 자료 다운로드

In [2]:
import urllib.request
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip'
file_name = 'source_etl_proejct.zip'
urllib.request.urlretrieve(URL, file_name)

('source_etl_proejct.zip', <http.client.HTTPMessage at 0x1d74a022100>)

## **Importing Libraries and Setting Paths**  

이제 프로젝트 폴더에 **필요한 파일들이 준비**되어 있습니다.

이 실습에서는 **CSV, JSON, XML 형식의 데이터**를 **추출(Extract)**할 것입니다. 먼저, 적절한 Python **라이브러리들을 가져오기(import)** 해야 관련 함수를 사용할 수 있습니다.

---

### 📚 **필요한 라이브러리 설명:**

1️⃣ **`xml` 라이브러리**:  
- **`.xml` 파일 형식**의 정보를 파싱(parse)하는 데 사용됩니다.

2️⃣ **`pandas` 라이브러리**:  
- **`.csv`** 및 **`.json`** 파일 형식을 읽을 수 있습니다.  
- 데이터를 **데이터프레임(DataFrame)** 형식으로 변환하여 저장할 때 사용됩니다.

3️⃣ **`glob` 라이브러리**:  
- **파일 경로**와 **파일 형식** 정보를 가져올 때 사용됩니다.  
- 특정 디렉토리에 있는 **모든 파일을 검색**할 수 있습니다.

4️⃣ **`datetime` 라이브러리**:  
- **로그(logging)** 시점에 **날짜와 시간** 정보를 기록할 때 필요합니다.

---

### 🔧 **라이브러리 설치:**
- **`glob`**, **`xml`**, **`datetime`**은 **Python 내장 라이브러리**이므로 별도의 설치가 필요하지 않습니다.  
- 그러나 **`pandas`**는 별도로 설치해야 합니다.

👉 **pandas 설치 명령어**:  
```bash
pip install pandas
```

---

### ✅ **전체 코드 예시:**

```python
import pandas as pd
import xml.etree.ElementTree as ET
import glob
from datetime import datetime

# 파일 경로 설정
path = "/home/project/"
```

이제 **필요한 라이브러리**를 가져오고 **경로 설정**이 완료되었습니다. 😊

<div class='alert alert-block alert-warning'>

### **📌 주의:**  
XML 파일을 파싱할 때, **`xml.etree` 라이브러리에서 `ElementTree` 함수만 import**합니다.  
이 함수는 **XML 데이터를 파싱**하는 데 필요하기 때문입니다.

---

### 🔧 **글로벌 경로 설정:**  
코드 내 모든 함수에서 사용될 두 개의 **파일 경로**를 **글로벌 변수**로 설정해야 합니다:

1. **`transformed_data.csv`**:  
   - **최종 출력 데이터**를 저장할 파일로, **데이터베이스에 적재(load)**할 준비가 된 데이터를 담습니다.  

2. **`log_file.txt`**:  
   - 모든 **로그(log)** 정보를 저장하는 파일로 사용됩니다.

---

### ✅ **경로 설정 코드 예시:**

```python
# 필요한 라이브러리 가져오기
import pandas as pd
import xml.etree.ElementTree as ET
import glob
from datetime import datetime

# 글로벌 경로 설정
transformed_data_path = "/home/project/transformed_data.csv"
log_file_path = "/home/project/log_file.txt"
```

이제 **데이터 저장 경로**와 **로그 파일 경로**가 설정되었습니다. 😊


</div>

##  **Task 1: Extraction - 구현 요약**

다양한 파일 형식(CSV, JSON, XML)에서 데이터를 추출하는 **ETL(Extract, Transform, Load)** 과정의 **추출(Extraction)** 부분을 구현합니다. 이 과정에서는 각 파일 형식에 대해 **별도의 함수**를 정의하고, 모든 파일을 처리하는 **`extract()` 함수**를 작성합니다.

---

### 📄 **1️⃣ CSV 파일에서 데이터 추출: `extract_from_csv()`**  
CSV 파일을 읽기 위해 **`pandas.read_csv()`** 함수를 사용합니다.

```python
import pandas as pd

def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process) 
    return dataframe
```

---

### 📄 **2️⃣ JSON 파일에서 데이터 추출: `extract_from_json()`**  
JSON 파일을 읽기 위해 **`pandas.read_json()`** 함수를 사용하며, **`lines=True`** 인수를 추가하여 각 줄을 JSON 객체로 읽습니다.

```python
def extract_from_json(file_to_process): 
    dataframe = pd.read_json(file_to_process, lines=True) 
    return dataframe
```

---

### 📄 **3️⃣ XML 파일에서 데이터 추출: `extract_from_xml()`**  
XML 파일을 파싱하기 위해 **`ElementTree`**를 사용합니다. 이 예제에서는 **`name`**, **`height`**, **`weight`** 정보를 추출하고 **Pandas DataFrame**에 추가합니다.

```python
import pandas as pd
import xml.etree.ElementTree as ET

def extract_from_xml(file_to_process): 
    dataframe = pd.DataFrame(columns=["name", "height", "weight"]) 
    tree = ET.parse(file_to_process) 
    root = tree.getroot() 
    
    for person in root: 
        name = person.find("name").text 
        height = float(person.find("height").text) 
        weight = float(person.find("weight").text) 
        dataframe = pd.concat([dataframe, pd.DataFrame([{"name": name, "height": height, "weight": weight}])], ignore_index=True) 
    
    return dataframe
```

---

### 📄 **4️⃣ 모든 파일을 처리하는 함수: `extract()`**  
`glob` 라이브러리를 사용하여 **모든 CSV, JSON, XML 파일을 탐색**하고, 각 파일 형식에 맞는 함수를 호출하여 데이터를 추출합니다.

```python
import glob

def extract(): 
    extracted_data = pd.DataFrame(columns=['name', 'height', 'weight'])  # 빈 데이터프레임 생성
     
    # CSV 파일 처리
    for csvfile in glob.glob("*.csv"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True) 
         
    # JSON 파일 처리
    for jsonfile in glob.glob("*.json"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index=True) 
     
    # XML 파일 처리
    for xmlfile in glob.glob("*.xml"): 
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index=True) 
         
    return extracted_data
```

---

### ✅ **etl_code.py에 추가할 코드:**  
이제 위에서 작성한 모든 함수를 **`etl_code.py`** 파일에 추가하면 **추출(Extraction)** 부분의 구현이 완료됩니다.

- **`extract_from_csv()`**  
- **`extract_from_json()`**  
- **`extract_from_xml()`**  
- **`extract()`**

이제 데이터셋을 성공적으로 추출할 수 있습니다! 😊

## **Task 2: Transformation**  

데이터셋의 **`height`(키)** 값은 **인치(inches)** 단위로, **`weight`(몸무게)** 값은 **파운드(pounds)** 단위로 되어 있습니다.  
그러나, 응용 프로그램에서 요구하는 형식은:  
- **키(height)**: **미터(meters)**  
- **몸무게(weight)**: **킬로그램(kilograms)**  

따라서, 단위를 변환하고 **소수점 둘째 자리까지 반올림**하는 **`transform()` 함수**를 작성해야 합니다.

---

### ✅ **`transform()` 함수 설명:**  
- **키 변환:**  
  - **1 인치** = **0.0254 미터**  
- **몸무게 변환:**  
  - **1 파운드** = **0.45359237 킬로그램**  
- **소수점 둘째 자리**로 반올림(**`round()`** 함수 사용)

---

### 📄 **`transform()` 함수 코드:**

```python
def transform(data): 
    '''Convert inches to meters and round off to two decimals 
    1 inch is 0.0254 meters '''
    data['height'] = round(data.height * 0.0254, 2) 
 
    '''Convert pounds to kilograms and round off to two decimals 
    1 pound is 0.45359237 kilograms '''
    data['weight'] = round(data.weight * 0.45359237, 2) 
    
    return data
```

---

### ✅ **변환 후 데이터프레임 예시:**

| **name**  | **height (m)** | **weight (kg)** |
|-----------|-----------------|-----------------|
| John      | 1.75            | 70.31           |
| Alice     | 1.62            | 60.45           |
| Bob       | 1.80            | 78.91           |

---

### 🔧 **etl_code.py 파일에 추가:**  
- 위의 **`transform()` 함수**를 **`etl_code.py`** 파일에 추가하면 **데이터 변환(Transformation)** 단계가 완료됩니다. 😊

이제 **추출(Extraction)**과 **변환(Transformation)** 단계가 모두 구현되었습니다! 💪

## **Task 3: Loading and Logging**  

이제 변환된 데이터를 **CSV 파일로 저장(Loading)**하고, 각 단계의 진행 상황을 **로그(Log)로 기록**하는 기능을 구현합니다.

---

### ✅ **1️⃣ 데이터 적재 함수: `load_data()`**  
이 함수는 **변환된 데이터**를 **CSV 파일**로 저장하는 역할을 합니다.

📄 **`load_data()` 함수 코드:**  
```python
def load_data(target_file, transformed_data): 
    transformed_data.to_csv(target_file, index=False)  # index=False로 인덱스 저장 안 함
```

📌 **설명:**  
- **`target_file`**: 데이터를 저장할 **파일 경로**  
- **`transformed_data`**: **변환된 데이터프레임**  
- **`to_csv()`**: 데이터를 CSV 형식으로 저장하는 **Pandas 메서드**  
- **`index=False`**: 인덱스 정보를 파일에 저장하지 않도록 설정  

---

### ✅ **2️⃣ 로그 기록 함수: `log_progress()`**  
이 함수는 각 작업 단계의 **진행 상황**을 **로그 파일(log_file)**에 기록합니다.

📄 **`log_progress()` 함수 코드:**  
```python
from datetime import datetime

def log_progress(message): 
    timestamp_format = '%Y-%b-%d-%H:%M:%S'  # 연-월-일-시:분:초 형식
    now = datetime.now()  # 현재 시간 가져오기
    timestamp = now.strftime(timestamp_format)  # 문자열 형식으로 변환
    
    with open(log_file_path, "a") as f:  # 로그 파일을 추가 모드로 열기
        f.write(timestamp + ',' + message + '\n')  # 로그 메시지 기록
```

📌 **설명:**  
- **`message`**: 로그에 기록할 **메시지**  
- **`timestamp_format`**: 날짜 및 시간 형식 정의  
- **`strftime()`**: **날짜/시간을 문자열**로 변환  
- **`with open(log_file_path, "a")`**: 로그 파일을 **추가 모드**로 열고, 로그 메시지를 기록  

---

### ✅ **3️⃣ 모든 함수 통합하기:**  
- **`load_data()`**와 **`log_progress()`** 함수를 **`etl_code.py`** 파일에 추가합니다.  
- 이로써 **ETL(Extract, Transform, Load)**의 모든 기능이 완성되었습니다.

---

### 📄 **etl_code.py 최종 코드 구조:**  
```python
import pandas as pd
import xml.etree.ElementTree as ET
import glob
from datetime import datetime

# 경로 설정
transformed_data_path = "/home/project/transformed_data.csv"
log_file_path = "/home/project/log_file.txt"

# Extract functions (CSV, JSON, XML)
def extract_from_csv(file_to_process):
    return pd.read_csv(file_to_process)

def extract_from_json(file_to_process):
    return pd.read_json(file_to_process, lines=True)

def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = pd.concat([dataframe, pd.DataFrame([{"name": name, "height": height, "weight": weight}])], ignore_index=True)
    return dataframe

# Transformation function
def transform(data):
    data['height'] = round(data.height * 0.0254, 2)
    data['weight'] = round(data.weight * 0.45359237, 2)
    return data

# Loading function
def load_data(target_file, transformed_data):
    transformed_data.to_csv(target_file, index=False)

# Logging function
def log_progress(message):
    timestamp_format = '%Y-%b-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(log_file_path, "a") as f:
        f.write(timestamp + ',' + message + '\n')
```

---

### ✅ **ETL 프로세스 테스트 준비 완료!**  
이제 모든 **ETL 함수(Extract, Transform, Load)**가 구현되었으며,  
**`etl_code.py`** 파일에 추가한 후 **테스트 실행**을 통해 ETL 프로세스를 확인할 수 있습니다. 🎉

### **Execution of Code (코드 실행)**  

`etl_code.py` 파일을 **터미널 쉘**에서 다음 명령어를 사용하여 실행합니다:  

```bash
python3.11 etl_code.py
```  

### ✅ **코드 실행 결과:**  
- 실행이 완료되면, **터미널에서 출력 메시지**를 확인할 수 있습니다.  
- 각 작업 단계에서 **`print` 명령어**로 출력된 결과가 터미널에 표시됩니다.

<img src='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/images/code_output.png' />



---

### 📄 **로그 파일(log_file.txt) 내용 확인:**  
코드 실행 후, **`log_file.txt`** 파일을 열면 로그 파일에 기록된 **진행 상황**을 확인할 수 있습니다.

<img src='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/images/log_file.png' />

---

### ✅ **실행 흐름 요약:**
1. **터미널에서 `etl_code.py` 실행**  
2. **터미널 출력 확인 (print 명령어 결과)**  
3. **`log_file.txt` 확인 (각 단계의 작업 기록)**

이제 ETL 프로세스가 성공적으로 실행되었습니다! 🎉

<hr>

# Practice Exercises

In [41]:
import urllib.request
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip '
file_name = 'datasource.zip'
urllib.request.urlretrieve(URL, file_name)

('datasource.zip', <http.client.HTTPMessage at 0x1d75b6d0280>)

### ✅ **Practice Exercise Guide:**

아래는 **ETL 실습 과제**를 해결하기 위한 **단계별 가이드**입니다. 각 단계를 따라가며 **CSV, JSON, XML** 형식의 데이터를 추출, 변환, 적재, 로그 기록하는 기능을 구현하세요.

---

### 📂 **1️⃣ 디렉토리 및 파일 생성:**

1. **데이터 소스 폴더 생성:**  
   ```bash
   mkdir -p /home/project/data_source
   ```

2. **디렉토리 변경:**  
   ```bash
   cd /home/project/data_source
   ```

3. **새 Python 파일 생성:**  
   ```bash
   touch etl_practice.py
   ```

---

### 📦 **2️⃣ 데이터 다운로드 및 압축 해제:**

- 다운로드한 데이터를 **data_source** 폴더에 저장하고 **압축을 해제**합니다.

---

### 🧑‍💻 **3️⃣ etl_practice.py 파일 작성:**

**etl_practice.py**에 아래 기능들을 단계별로 구현하세요.

---

### 📄 **Step 1: Extraction (데이터 추출)**

각 파일 형식에 대해 데이터를 추출하는 함수를 작성합니다:

- **CSV:** `extract_from_csv()`
- **JSON:** `extract_from_json()`
- **XML:** `extract_from_xml()`

```python
import pandas as pd
import xml.etree.ElementTree as ET
import glob

def extract_from_csv(file_to_process):
    return pd.read_csv(file_to_process)

def extract_from_json(file_to_process):
    return pd.read_json(file_to_process, lines=True)

def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["car_model", "year_of_manufacture", "price", "fuel"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for car in root:
        car_model = car.find("car_model").text
        year_of_manufacture = int(car.find("year_of_manufacture").text)
        price = float(car.find("price").text)
        fuel = car.find("fuel").text
        dataframe = pd.concat([dataframe, pd.DataFrame([{"car_model": car_model, "year_of_manufacture": year_of_manufacture, "price": price, "fuel": fuel}])], ignore_index=True)
    return dataframe
```

---

### 📄 **Step 2: Transformation (데이터 변환)**

`price` 값을 **소수점 둘째 자리**로 반올림하는 변환 작업을 수행합니다.

```python
def transform(data):
    data['price'] = round(data['price'], 2)
    return data
```

---

### 📄 **Step 3: Loading (데이터 적재)**

변환된 데이터를 **transformed_data.csv** 파일로 저장합니다.

```python
def load_data(target_file, transformed_data):
    transformed_data.to_csv(target_file, index=False)
```

---

### 📄 **Step 4: Logging (로그 기록)**

각 단계의 진행 상황을 **log_file.txt**에 기록합니다.

```python
from datetime import datetime

def log_progress(message):
    timestamp_format = '%Y-%b-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("log_file.txt", "a") as f:
        f.write(timestamp + "," + message + "\n")
```

---

### 🧪 **Step 5: Testing (테스트)**

전체 ETL 프로세스를 테스트합니다.

```python
def extract():
    extracted_data = pd.DataFrame(columns=["car_model", "year_of_manufacture", "price", "fuel"])

    for csvfile in glob.glob("*.csv"):
        extracted_data = pd.concat([extracted_data, extract_from_csv(csvfile)], ignore_index=True)

    for jsonfile in glob.glob("*.json"):
        extracted_data = pd.concat([extracted_data, extract_from_json(jsonfile)], ignore_index=True)

    for xmlfile in glob.glob("*.xml"):
        extracted_data = pd.concat([extracted_data, extract_from_xml(xmlfile)], ignore_index=True)

    return extracted_data

if __name__ == "__main__":
    log_progress("ETL process started")

    # Extraction
    data = extract()
    log_progress("Extraction completed")

    # Transformation
    transformed_data = transform(data)
    log_progress("Transformation completed")

    # Loading
    load_data("transformed_data.csv", transformed_data)
    log_progress("Loading completed")

    log_progress("ETL process finished")
```

---

### ✅ **실행 방법:**

1. **터미널에서 etl_practice.py 실행:**  
   ```bash
   python3 etl_practice.py
   ```

2. **로그 파일 확인:**  
   ```bash
   cat log_file.txt
   ```

---

### 🎉 **완료!**

이제 모든 ETL 단계를 구현하고 테스트를 완료했습니다!  
궁금한 점이 있으면 **Discussion Forum**에 질문을 올려보세요. 😊